<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-14 10:33:40
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.36 C
-------------------
Today PnL: -458.00 (-0.0%)
Current PnL: -13.99 L (-9.75%)
CY Booked + Current PnL: -7.03 L (-4.9%)
-------------------
Total profit:  4.70 L
Total loss:  -18.69 L
-------------------
Total Booked + Current PnL: 19.95 L (17.12%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.11%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 72.53 L (53.28%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.35%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-220.24,H-SC,79.17,86736.0,-8580.0,8622.0,-2.57,-9.00,9.94,0.04,157.0,-1.00,0.64,71.08,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.00,52.86,M-SC,56.25,132246.0,-10233.0,10857.0,3.03,-7.18,8.21,0.44,247.0,-0.94,0.97,56.09,OX40N,NTT,BANKS
52,JPPOWER,18.73,26.20,-8.11,L-SC,100.00,185008.0,43166.0,13413.0,3.39,30.43,7.25,39.88,262.0,3.22,1.36,87.49,XY24,NTT,POWER
71,SBIN,760.30,863.00,-11.51,M-LC,63.54,215233.0,13754.0,13452.0,0.44,6.83,6.25,13.51,77.0,1.02,1.58,20.28,XY25,NTT,BANKS
39,HINDALCO,651.95,761.55,-21.55,H-LC,72.92,108208.0,3896.0,13645.0,0.80,3.73,12.61,16.81,27.0,0.29,0.80,20.38,X5K,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BERGEPAINT,561.33,680.00,-19.35,H-MC,40.10,228339.0,1000.0,47061.0,-1.53,0.44,20.61,21.14,117.0,0.02,1.68,27.85,XY24,NTT,PAINTS
60,NATIONALUM,189.63,247.44,-49.40,H-MC,59.38,101474.0,970.0,29671.0,1.03,0.97,29.24,30.49,97.0,0.03,0.75,33.68,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-8.11,L-SC,100.00,185008.0,43166.0,13413.0,3.39,30.43,7.25,39.88,262.0,3.22,1.36,87.49,XY24,NTT,POWER
87,UJJIVANSFB,52.77,53.00,52.86,M-SC,56.25,132246.0,-10233.0,10857.0,3.03,-7.18,8.21,0.44,247.0,-0.94,0.97,56.09,OX40N,NTT,BANKS
45,INDIAMART,2327.09,4911.36,-45.65,H-SC,85.42,141234.0,17898.0,119074.0,2.31,14.51,84.31,111.05,123.0,0.15,1.04,38.38,AR,ATH,MISC
41,HINDZINC,514.80,730.22,15.78,H-LC,1.04,86940.0,-16020.0,59102.0,2.31,-15.56,67.98,41.85,24.0,-0.27,0.64,14.42,X5K,ATH,METALS
27,DEN,52.79,75.00,122.80,L-SC,87.50,62464.0,-22000.0,57536.0,2.17,-26.05,92.11,42.07,261.0,-0.38,0.46,32.65,AR,NTT,ENTERTAINMENT


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.7,NaN,NaN,94.79,76355.0,41906.0,3062.0,-3.86,121.65,4.01,130.54,274.0,13.69,0.56,16.76,AR,ATH,ELECTRICAL
92,VIPIND,488.80,489.0,-220.24,H-SC,79.17,86736.0,-8580.0,8622.0,-2.57,-9.00,9.94,0.04,157.0,-1.00,0.64,71.08,OX40N,NTT,MISC
63,QUESS,296.83,986.0,-32.78,M-SC,27.08,66795.0,1789.0,149140.0,-1.83,2.75,223.28,232.18,193.0,0.01,0.49,2.28,XY24,NTT,MISC
14,BERGEPAINT,561.33,680.0,-19.35,H-MC,40.10,228339.0,1000.0,47061.0,-1.53,0.44,20.61,21.14,117.0,0.02,1.68,27.85,XY24,NTT,PAINTS
19,CAMPUS,294.86,393.0,-23.81,M-SC,36.46,158808.0,-8967.0,64810.0,-1.52,-5.34,40.81,33.28,206.0,-0.14,1.17,22.95,XY24,NTT,FOOTWEAR


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,52.86,M-SC,56.25,132246.0,-10233.0,10857.0,3.03,-7.18,8.21,0.44,247.0,-0.94,0.97,56.09,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-220.24,H-SC,79.17,86736.0,-8580.0,8622.0,-2.57,-9.00,9.94,0.04,157.0,-1.00,0.64,71.08,OX40N,NTT,MISC
79,SYMPHONY,1306.42,1306.0,-29.94,M-SC,47.92,149406.0,-21735.0,21679.0,-0.77,-12.70,14.51,-0.03,191.0,-1.00,1.10,7.38,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,93.75,148924.0,-25635.0,25674.0,0.07,-14.69,17.24,0.02,133.0,-1.00,1.09,29.14,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.65,X-MC,19.79,86394.0,-16258.0,16251.0,1.73,-15.84,18.81,-0.00,59.0,-1.00,0.64,31.33,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-22.56,X-LC,5.21,252728.0,-43206.0,137105.0,-0.79,-14.60,54.25,31.73,1.0,-0.32,1.86,1.16,X40,BTT,IT
48,INFY,1461.46,2275.00,-12.06,X-LC,20.83,281961.0,20360.0,125275.0,-1.24,7.78,44.43,55.67,2.0,0.16,2.07,14.36,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-10.23,X-LC,90.62,281030.0,10683.0,23944.0,-0.41,3.95,8.52,12.81,5.0,0.45,2.07,17.53,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,15.41,X-LC,68.75,264546.0,35313.0,31984.0,-0.13,15.40,12.09,29.35,10.0,1.10,1.94,25.70,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-7.18,X-LC,12.50,78375.0,-31797.0,120870.0,0.55,-28.86,154.22,80.85,11.0,-0.26,0.58,15.43,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.7,NaN,NaN,94.79,76355.0,41906.0,3062.0,-3.86,121.65,4.01,130.54,274.0,13.69,0.56,16.76,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.2,-8.11,L-SC,100.00,185008.0,43166.0,13413.0,3.39,30.43,7.25,39.88,262.0,3.22,1.36,87.49,XY24,NTT,POWER
59,MEDANTA,1064.65,1486.0,18.72,X-SC,89.58,253312.0,47835.0,33488.0,0.71,23.28,13.22,39.58,65.0,1.43,1.86,35.74,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,94.40,M-MC,78.12,232866.0,7904.0,159746.0,0.67,3.51,68.60,74.53,189.0,0.05,1.71,35.20,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.01,31.10,H-SC,84.38,133976.0,8679.0,50496.0,2.00,6.93,37.69,47.22,143.0,0.17,0.98,67.61,MH,ATH,POWER
20,CAMS,3643.00,5250.99,2.54,H-SC,50.00,115195.0,13191.0,31828.0,1.09,12.93,27.63,44.14,126.0,0.41,0.85,31.27,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,-4.05,H-SC,86.46,180110.0,15440.0,53637.0,0.07,9.38,29.78,41.95,124.0,0.29,1.32,17.54,X40N,ATH,AC
88,UNIONBANK,123.87,163.00,-4.47,M-LC,48.96,166787.0,25947.0,18547.0,2.05,18.42,11.12,31.59,88.0,1.40,1.23,44.88,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,94.79,76355.0,41906.0,3062.0,-3.86,121.65,4.01,130.54,274.0,13.69,0.56,16.76,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.43,M-SC,76.04,202919.0,31298.0,130781.0,0.58,18.24,64.45,94.43,201.0,0.24,1.49,49.77,XR,NTT,DURABLES
54,KPIGREEN,497.21,732.01,31.10,H-SC,84.38,133976.0,8679.0,50496.0,2.00,6.93,37.69,47.22,143.0,0.17,0.98,67.61,MH,ATH,POWER
45,INDIAMART,2327.09,4911.36,-45.65,H-SC,85.42,141234.0,17898.0,119074.0,2.31,14.51,84.31,111.05,123.0,0.15,1.04,38.38,AR,ATH,MISC
95,WIPRO,243.46,420.00,-7.40,M-LC,21.88,158255.0,7310.0,102138.0,-1.18,4.84,64.54,72.51,71.0,0.07,1.16,9.01,XR,NTT,IT


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.36,NaN,NaN,96.88,18008.0,-31585.0,904.0,-0.32,-63.69,5.02,-61.86,270.0,-34.94,0.13,20.77,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,43.75,66816.0,-46733.0,86787.0,0.03,-41.16,129.89,35.27,265.0,-0.54,0.49,63.53,XR,NTT,HOTELS
6,ASIANTILES,75.41,137.00,7211.11,M-SC,31.25,80226.0,-13584.0,90206.0,0.02,-14.48,112.44,81.67,245.0,-0.15,0.59,55.36,XR,NTT,CERAMICS
92,VIPIND,488.80,489.00,-220.24,H-SC,79.17,86736.0,-8580.0,8622.0,-2.57,-9.00,9.94,0.04,157.0,-1.00,0.64,71.08,OX40N,NTT,MISC
58,MASFIN,326.60,399.50,-18.25,H-SC,82.29,92745.0,-5235.0,27109.0,-0.31,-5.34,29.23,22.32,145.0,-0.19,0.68,34.51,XR,ATH,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-8.11,L-SC,100.00,185008.0,43166.0,13413.0,3.39,30.43,7.25,39.88,262.0,3.22,1.36,87.49,XY24,NTT,POWER
65,RELAXO,902.64,1176.00,-30.09,H-SC,98.96,63194.0,-52344.0,87334.0,-1.42,-45.30,138.20,30.28,131.0,-0.60,0.46,23.46,X40N,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,3.90,X-SC,97.92,96754.0,-4209.0,98737.0,0.71,-4.17,102.05,93.63,67.0,-0.04,0.71,44.11,SR,ATH,CHEMICALS
16,BLUSPRING,226.45,86.36,NaN,NaN,96.88,18008.0,-31585.0,904.0,-0.32,-63.69,5.02,-61.86,270.0,-34.94,0.13,20.77,XY24,ATH,MISC
26,DABUR,505.20,735.00,-4.67,X-MC,95.83,206388.0,9360.0,80264.0,-0.31,4.75,38.89,45.49,56.0,0.12,1.52,17.24,XY24,BTT,FMCG


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.93
1,25,41.85
2,50,70.61


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.86
LC    33.47
MC    21.64
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.22
X40      12.76
XY25     12.11
X40N     11.84
XR        9.96
OX40N     8.23
AR        8.18
MH        1.73
X5K       1.44
X200      1.36
SR        1.19
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.56
M-SC    15.95
H-LC    13.92
X-LC    12.73
H-MC     9.58
X-MC     7.63
M-LC     5.78
X-SC     4.58
M-MC     4.07
L-SC     2.77
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.26,-0.82,34.52
IT,11.83,-12.93,68.43
FINANCE,8.47,-13.12,54.17
BANKS,7.73,-8.43,52.67
MISC,5.92,-19.59,75.69
PAINTS,5.61,-14.32,36.85
ELECTRICAL,5.44,0.32,46.92
HEALTHCARE,4.76,0.53,28.52
AUTO,4.26,-16.71,68.04


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2643763.0,26
XR,1103132.0,13
AR,1040388.0,10
X40,720630.0,10
X40N,550786.0,11
XY25,472113.0,8
OX40N,311770.0,11
SR,182393.0,2
MH,80167.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1988374.0,20
M-SC,1774998.0,21
H-LC,647013.0,12
X-LC,578329.0,8
H-MC,532064.0,8
X-MC,416419.0,6
M-MC,415083.0,3
X-SC,296432.0,5
M-LC,274230.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      897033.0      6
M-SC       XY24      809651.0      7
H-SC       AR        520594.0      3
M-SC       XR        336365.0      4
H-MC       XY24      302782.0      3
M-MC       XY24      300149.0      2
X-LC       X40       299360.0      3
H-SC       XR        222852.0      3
           X40N      208442.0      4
H-LC       X40N      206267.0      4
M-SC       AR        189058.0      2
           OX40N     165447.0      5
H-LC       X40       163526.0      3
M-LC       XY24      158640.0      2
X-MC       X40       157927.0      2
L-SC       XR        150107.0      2
X-SC       XR        123092.0      1
X-LC       AR        120870.0      1
M-SC       XY25      119736.0      1
M-MC       XR        114934.0      1
X-MC       XY25      108873.0      1
M-LC       XR        102138.0      1
X-LC       XY25       99787.0      3
X-SC       SR         98737.0      1
H-SC       OX40N      88957.0      3
H-LC       AR         85709.0      1
M-SC       SR         83656.0      1
H-MC       XY25       82659.0      1
X-MC       XY24       80264.0      1
H-LC       X200       74767.0      1
           X5K        72747.0      2
M-SC       X40        71085.0      1
H-MC       AR         63559.0      1
X-LC       X40N       58312.0      1
L-SC       AR         57536.0      1
L-MC       XR         53644.0      1
X-MC       X40N       53104.0      1
H-SC       MH         50496.0      1
L-LC       XY25       47606.0      1
H-LC       XY24       43997.0      1
X-SC       OX40N      41115.0      2
           XY24       33488.0      1
H-MC       MH         29671.0      1
           X40        28732.0      1
           X40N       24661.0      1
X-MC       OX40N      16251.0      1
M-LC       XY25       13452.0      1
L-SC       XY24       13413.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
